In [1]:
# First we have to fetch the env variables created in .env

import os

from dotenv import load_dotenv

load_dotenv()

True

In [4]:
# Now let's get and set the envs here

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

# LangSmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [11]:
# Now let's load the openai model in langchain

from langchain_openai import ChatOpenAI

# Now let's get a OpenAI model to work with

llm = ChatOpenAI(model="gpt-4o-mini")

llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001B4DC754CE0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001B4D6DCBDA0>, root_client=<openai.OpenAI object at 0x000001B4D65FBEC0>, root_async_client=<openai.AsyncOpenAI object at 0x000001B4DC755370>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [12]:
# Now let's simply call the OpenAI API and ask a question

response=llm.invoke("Tell me about agentic AI")

# Once this will be executed then LangSmith will start tracing our LLM calls happening

response

AIMessage(content='Agentic AI refers to artificial intelligence systems that exhibit a degree of agency, meaning they can act independently to achieve specific goals or objectives. These systems not only process information and provide outputs based on predefined algorithms but also demonstrate a level of autonomy in decision-making and action-taking. Here are some key aspects of agentic AI:\n\n### 1. **Definition of Agency**:\n   - **Autonomy**: Agentic AI can make decisions without human intervention, based on its programming and learning.\n   - **Goal-directed behavior**: It’s designed to pursue specific goals and can adjust its actions based on changing circumstances.\n\n### 2. **Examples**: \n   - **Autonomous Vehicles**: These systems make real-time decisions about navigation, obstacle avoidance, and speed control.\n   - **Robotics**: Robots that can operate autonomously in manufacturing, healthcare, or service industries.\n   - **Smart Assistants**: Some AI tools can operate ind

In [15]:
# Now let's ask the llm to behave in a certain way by setting up the prompt

from langchain_core.prompts import ChatPromptTemplate


prompt=ChatPromptTemplate.from_messages(
    [("system","You are an expert AI Engineer. Answer the questions asked by the user best of your knowledge"),
     ("user", "{input}")]
)

# Here the system message is the prompt given to llm to behave in a certain way
# user is the user message that the user will ask as question

prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Answer the questions asked by the user best of your knowledge'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [16]:
# Now we have rthe prompt ready. Now when the user asks any query it will come to the prompt and then it will go to llm
# So this process needs to be combined which can be done using chains provided by langchain


chain=prompt|llm

response=chain.invoke({"input":"Tell me about langsmith"})
response

AIMessage(content='Langsmith is a platform that focuses on enhancing the development and deployment of AI models, particularly in natural language processing (NLP) applications. It is designed to assist developers and organizations in building, evaluating, and optimizing language models effectively. Langsmith often includes features such as data management, model training, evaluation metrics, and deployment tools to streamline the workflow involved in working with language models.\n\nWhile specific details about Langsmith may evolve over time, these platforms typically aim to make it easier for users to understand and iterate on their NLP projects, providing infrastructure and tools that support collaboration and scalability.\n\nIf you have specific aspects or functionalities of Langsmith you would like to learn more about, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 141, 'prompt_tokens': 34, 'total_tokens': 175, 'com

In [17]:
print(response)

content='Langsmith is a platform that focuses on enhancing the development and deployment of AI models, particularly in natural language processing (NLP) applications. It is designed to assist developers and organizations in building, evaluating, and optimizing language models effectively. Langsmith often includes features such as data management, model training, evaluation metrics, and deployment tools to streamline the workflow involved in working with language models.\n\nWhile specific details about Langsmith may evolve over time, these platforms typically aim to make it easier for users to understand and iterate on their NLP projects, providing infrastructure and tools that support collaboration and scalability.\n\nIf you have specific aspects or functionalities of Langsmith you would like to learn more about, feel free to ask!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 141, 'prompt_tokens': 34, 'total_tokens': 175, 'completion_toke

In [20]:
# In the above line we can see we have a content field which we want to get rid of and want a generic text only output
# For this we can user stroutputparser

from langchain_core.output_parsers import StrOutputParser

outputParser=StrOutputParser()

chain=prompt|llm|outputParser

response=chain.invoke({"input":"Tell me about langsmith"})
response

"LangSmith is a platform designed to facilitate the development and deployment of AI applications, particularly those involving natural language processing (NLP). By providing tools and frameworks, LangSmith aims to streamline the process of building conversational agents, chatbots, and other language-based applications.\n\nKey features of LangSmith often include:\n\n1. **User-Friendly Interface**: LangSmith typically offers a user-friendly interface that allows developers, even those with limited coding experience, to build and manage their NLP models easily.\n\n2. **Framework Support**: The platform usually supports popular machine learning and NLP frameworks, making it easier for developers to leverage existing models or create new ones.\n\n3. **Integration Capabilities**: LangSmith might provide integration options with various messaging platforms and APIs, enabling seamless deployment of language applications across different channels.\n\n4. **Collaboration Tools**: The platform m

# Json output parser

In [21]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

json_output=JsonOutputParser()

prompt=PromptTemplate(
    template="Provide the response in \n{output_template} for the query \n{query}\n",
    input_variables=["query"],
    partial_variables={"output_template":json_output.get_format_instructions()}
)

In [22]:
chain2=prompt|llm|json_output

response2=chain2.invoke({"query":"Tell me difference between agent and agentic workflows"})
response2

{'difference': {'agent_workflow': {'definition': 'An agent workflow refers to processes or tasks that are primarily executed by a human agent or operator, often requiring human decision-making and intervention.',
   'characteristics': ['Dependent on human intervention',
    'Involves subjective decision-making',
    'Can adapt to unstructured or unexpected situations',
    'Typically includes communication with other humans']},
  'agentic_workflow': {'definition': 'An agentic workflow refers to processes that can be partially or fully automated, where intelligent systems or software agents perform tasks with some level of autonomy.',
   'characteristics': ['Higher level of automation',
    'Ability to operate independently',
    'Uses predefined rules and algorithms',
    'Seeks to minimize human involvement in routine tasks']}}}

### Explore all different kinds of output parsers available in langchain

### Go through the langchain documentation very important

# RAG

# We will be using the llm_chain documentation page from langchain



In [25]:
from langchain.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [26]:
loader = WebBaseLoader("https://python.langchain.com/docs/tutorials/llm_chain/")

In [27]:
document=loader.load()
document

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!", 'language': 'en'}, page_content='\n\n\n\n\nBuild a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuil

In [28]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

### Now we have to create chunks out of the document

In [29]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

documents=splitter.split_documents(document)

documents

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!", 'language': 'en'}, page_content='Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain.

### We have to convert these chunks into vectors and store them into vectoreStore DB

In [30]:
from langchain.embeddings import OpenAIEmbeddings

embedding=OpenAIEmbeddings()

C:\Users\shubh\AppData\Local\Temp\ipykernel_8944\2058545845.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding=OpenAIEmbeddings()


In [32]:
# Now we will use FAISS as the vector Store DB for us and will store the embeddings there

from langchain_community.vectorstores import FAISS

vectorStore=FAISS.from_documents(documents=documents, embedding=embedding)

vectorStore

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

#### Let's do similarity search

In [33]:
results=vectorStore.similarity_search("This is realtively simple application")

results[0].page_content

NameError: name 'vectorStore' is not defined

#### Now let's use llm to format the retrieved info and display in a meaningful way

In [36]:
# Here we will be using from_template as the the prompt will not be having any stsyem message or user message
# as we are getting the info from the FAISS vector store

prompts=ChatPromptTemplate.from_template(
    """ Please take the below context and formulate the response.
    <context>
    {context}
    </context>
"""
)

In [39]:
# Now let's create the chain and get the output

from langchain.chains.combine_documents import create_stuff_documents_chain # It is used as we are getting the output from similarity and then stuffing it to LLM


document_chain=create_stuff_documents_chain(llm, prompts)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' Please take the below context and formulate the response.\n    <context>\n    {context}\n    </context>\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001B4DC754CE0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001B4D6DCBDA0>, root_client=<openai.OpenAI object at 0x000001B4D65FBEC0>, root_async_client=<openai.AsyncOpenAI object at 0x000001B4DC755370>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, c

# Check for all the document loaders available in the langchain documentation
   https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/
# Check for all the text splitters available in the langchain documentation
   https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/
# Check for all the embedding models available in the langchain documentation
   https://python.langchain.com/docs/integrations/text_embedding/
# Check for all the vector stores available in the langchain documentation
   https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/